In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyodbc
import sqlite3
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
# slesConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=./go_sales_schoon.accdb;')

import pyodbc
import sqlite3

def convert_accdb_to_sqlite(accdb_file, sqlite_file):
    # Connect to the Access database
    accdb_conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + accdb_file
    accdb_conn = pyodbc.connect(accdb_conn_str)
    accdb_cursor = accdb_conn.cursor()

    # Get the table names from the Access database
    table_names = []
    for row in accdb_cursor.tables():
        if row.table_type == 'TABLE':
            table_names.append(row.table_name)

    # Connect to the SQLite database
    sqlite_conn = sqlite3.connect(sqlite_file)
    sqlite_cursor = sqlite_conn.cursor()

    # Iterate through each table and copy the data to SQLite
    for table_name in table_names:
        # Retrieve the table data from Access
        accdb_cursor.execute(f"SELECT * FROM [{table_name}]")
        table_data = accdb_cursor.fetchall()

        # Get the column names from Access
        accdb_cursor.execute(f"SELECT * FROM [{table_name}]")
        column_names = [column[0] for column in accdb_cursor.description]

        # Create the table in SQLite without specifying column data types
        create_table_query = f'CREATE TABLE "{table_name}" ' + "(" + ", ".join([f'"{col}"' for col in column_names]) + ")"
        sqlite_cursor.execute(create_table_query)

        # Generate placeholders based on the number of columns
        placeholders = ",".join(["?" for _ in column_names])

        # Insert the data into the SQLite table, converting column types if necessary
        sqlite_cursor.executemany(f'INSERT INTO "{table_name}" VALUES ({placeholders})', table_data)

    # Commit the changes and close the connections
    sqlite_conn.commit()
    sqlite_conn.close()
    accdb_conn.close()

    print("Conversion completed successfully.")



# Usage example
convert_accdb_to_sqlite("./go_staff_schoon.accdb;", "staff.sqlite")


In [1]:
import sqlite3
import shutil

def copy_tables_to_single_file(source_files, destination_file):
    # Create a new SQLite database file
    conn_dest = sqlite3.connect(destination_file)
    cursor_dest = conn_dest.cursor()

    for source_file in source_files:
        # Connect to the source database file
        conn_source = sqlite3.connect(source_file)
        cursor_source = conn_source.cursor()

        # Get the list of tables in the source database
        cursor_source.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor_source.fetchall()

        # Copy each table to the destination database
        for table in tables:
            table_name = table[0]

            # Retrieve the table structure from the source database
            cursor_source.execute(f"PRAGMA table_info([{table_name}])")
            table_info = cursor_source.fetchall()

            # Create the table in the destination database if it doesn't exist
            create_table_query = f"CREATE TABLE IF NOT EXISTS [{table_name}] ("
            for column in table_info:
                column_name = column[1]
                column_type = column[2]
                create_table_query += f"[{column_name}] {column_type}, "
            create_table_query = create_table_query[:-2] + ");"

            cursor_dest.execute(create_table_query)

            # Get the unique identifier column name (assuming it's the first column)
            unique_column = table_info[0][1]

            # Copy the data from the source table to the destination table
            cursor_source.execute(f"SELECT * FROM [{table_name}];")
            rows = cursor_source.fetchall()
            for row in rows:
                # Check if the row already exists in the destination table
                cursor_dest.execute(
                    f"SELECT 1 FROM [{table_name}] WHERE [{unique_column}] = ?",
                    (row[0],)  # Assuming the unique column is the first column
                )
                result = cursor_dest.fetchone()

                if result is None:
                    # Generate the placeholders for the INSERT statement
                    placeholders = ",".join(["?" for _ in row])

                    # Insert the row into the destination table
                    cursor_dest.execute(f"INSERT INTO [{table_name}] VALUES ({placeholders})", row)

        # Commit the changes and close the source connection
        conn_dest.commit()
        conn_source.close()

    # Close the destination connection
    conn_dest.close()

# Example usage:
source_files = ["crm.sqlite","sales.sqlite","sales.sqlite"]
destination_file = "merged.sqlite"

copy_tables_to_single_file(source_files, destination_file)